<a href="https://colab.research.google.com/github/AkshayAkshay1/My-1/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install textstat

In [ ]:
import openpyxl
import string
import pandas as pd
import requests
from bs4 import BeautifulSoup
import textstat
import re
import math
from urllib.request import urlopen

stopwords_auditor = open('/content/StopWords_Auditor.txt', encoding="utf-8").read()
stopwords_Currencies = open('/content/StopWords_Currencies.txt',encoding="ISO-8859-1").read()
stopwords_datesandnumbers = open('/content/StopWords_DatesandNumbers.txt',encoding="utf-8").read()
StopWords_Generic = open('/content/StopWords_Generic.txt',encoding="utf-8").read()
StopWords_GenericLong = open('StopWords_GenericLong.txt',encoding="utf-8").read()
StopWords_Geographic = open('StopWords_Geographic.txt',encoding="utf-8").read()
StopWords_Names = open('StopWords_Names.txt',encoding="utf-8").read()
stopwords = stopwords_auditor+stopwords_Currencies+stopwords_datesandnumbers+StopWords_Generic+StopWords_GenericLong+StopWords_Geographic+StopWords_Names
negative_words = open('/content/negative-words.txt', encoding="ISO-8859-1").read()
positive_words = open('/content/positive-words.txt', encoding="ISO-8859-1").read()

workbook = openpyxl.load_workbook('Input.xlsx')
sheet = workbook['Sheet1']
Url= []
Url_id = []
for i in range(2,102):
  cell = sheet['B'+ str(i) ]
  Url.append(cell.value)
  cell = sheet['A'+ str(i)]
  Url_id.append(cell.value)

positiveScore = []
negativeScore = []
polarityScore = []
subjectivityScore = []
averageSentenceLength = []
percentageOfComplexWords = []
fogIndex = []
avgNoOfWordsPerSentence = []
complexWordsCount = []
wordCount = []
syllablePerWord = []
personalPronouns = []
averageWordLength = []
data = []


for url in Url:
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")
  element = soup.find("h1", class_="entry-title")
  if element is not None:
    text_title = element.get_text()
  element2 = soup.find("div", class_="td-post-content")
  if element2 is not None:
    text_content=element2.get_text()
  final_text = text_title+ ' ' + text_content


  lower_case = final_text.lower()
  cleaned_text = lower_case.translate(str.maketrans('\n', ' ', string.punctuation))
  tokenized_words = cleaned_text.split()
  final_words = []
  for word in tokenized_words:
      if word not in stopwords:
          final_words.append(word)

  def count_sentences(text):
    sentences = text.split( ".")
    return len(sentences)

  def pos_neg_pol_sub_score_avg_sen_len(final_word):
    positive_score = 0
    negative_score =0
    for word in final_word:
      if word in positive_words:
        positive_score = positive_score +1
      elif word in negative_words:
        negative_score = negative_score +1
    polarity_score = (positive_score-negative_score)/ ((positive_score+negative_score) + 0.000001)
    Subjectivity_score =  (positive_score + negative_score)/ ((len(final_word)) + 0.000001)
    Average_sentence_length = len(final_words)/count_sentences(final_text)
    return positive_score , negative_score , polarity_score , Subjectivity_score, Average_sentence_length


  Positive_score,Negative_score,Polarity_score,Subjectivity_Score,Average_sentence_length  = pos_neg_pol_sub_score_avg_sen_len(final_words)

  def count_1complex_words(text):
      words = text.split()
      complex_word_count = 0

      for word in words:
          syllable_count = textstat.syllable_count(word)

          if syllable_count >= 2:
              complex_word_count += 1
      return complex_word_count
  Percentage_of_complex_words = (count_1complex_words(str(final_words))/len(final_words))*100
  fog_index = 0.4* (Average_sentence_length+Percentage_of_complex_words)
  Average_number_of_words_per_sentence = (len(final_words)/count_sentences(final_text))
  def syllable_count(finalwords):
      words = finalwords
      total = 0
      counter = 0
      for word in words:
          syllable_count = textstat.syllable_count(word)
          total= total+syllable_count
          counter = counter+1
      Average_syllable_count_per_word = total/counter
      return Average_syllable_count_per_word
  Avg_syllable_count = syllable_count(final_words)
  pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
  pronouns = pronounRegex.findall(cleaned_text)

  def average_word_length(final):
    count=0
    for word in final:
      length = len(word)
      count = count + length
    average_word_lengths = count/len(final)
    return average_word_lengths


  Average_number_of_words_per_sentence = (len(final_words)/count_sentences(final_text))
  awl = average_word_length(final_words)
  count_of_complex_words = count_1complex_words(str(final_words))




  positiveScore.append(round(Positive_score,2))
  negativeScore.append(round(Negative_score,2))
  polarityScore.append(round(Polarity_score,2))
  subjectivityScore.append(round(Subjectivity_Score,2))
  averageSentenceLength.append(round(Average_sentence_length,2))
  percentageOfComplexWords.append(round(Percentage_of_complex_words,2))
  fogIndex.append(round(fog_index,2))
  avgNoOfWordsPerSentence.append(round(Average_number_of_words_per_sentence,2))
  complexWordsCount.append(round(count_of_complex_words,2))
  wordCount.append(round(len(final_words),2))
  syllablePerWord.append(round(Avg_syllable_count,2))
  personalPronouns.append(round(len(pronouns),2))
  averageWordLength.append(round(awl,2))


data.append(Url)
data.append( Url_id)
data.append(positiveScore)
data.append(negativeScore)
data.append(polarityScore)
data.append(subjectivityScore)
data.append(averageSentenceLength)
data.append(percentageOfComplexWords)
data.append(fogIndex)
data.append(avgNoOfWordsPerSentence)
data.append(complexWordsCount)
data.append(wordCount)
data.append(syllablePerWord)
data.append(personalPronouns)
data.append(averageWordLength)


pddf = pd.DataFrame(data).T
pddf.columns = ['URL', 'URL ID','Positive Score' , 'Negative Score' , 'Polarity Score','Subjectivity Score' , 'Average Sentence Length','Percentage of Complex Words', 'Fog Index','Avg No. Of Words/sentence','Complex Word Count', 'Word Count', 'Syllable Per Word', 'Personal Pronouns','Avg Word Length']
pddf.to_excel('output.xlsx', index=False)

